In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import openai

from dotenv import load_dotenv
import os
import tqdm

import time

from curl_cffi import requests
session = requests.Session(impersonate="chrome")

import cvxpy as cp

from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
choices = [15, 30, 45, 60]

In [ ]:
dotenv_path = os.path.join(os.path.dirname(os.getcwd()), '.env')
load_dotenv(dotenv_path)

def get_chatgpt_tickers(n):
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    message = f"Using a range of investing principles taken from leading funds, create a theoretical fund comprising of at least {n} stocks (mention their tickers) from the S&P500 with the goal to outperform the S&P500 index. Return as a list that can be parsed in python."
    
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": message}]
    )
    
    return response.choices[0].message.content

# Loop through each value and make 10 requests
with open(os.path.join("..", "data/chatgpt_tickers.txt"), "w") as f:
    for n in choices:
        for i in range(10):
            result = get_chatgpt_tickers(n)
            f.write(f"\n=== N={n} | Request {i+1} ===\n")
            f.write(result + "\n")
            print(f"[✔] Saved N={n}, Request {i+1}")
            time.sleep(1) 

In [ ]:
import ast

def extract_all_ticker_lists(filepath):
    request_blocks = {}
    current_key = None
    inside_list = False
    buffer = []

    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()

            # Detect a request header line
            if line.startswith("==="):
                current_key = line.strip("= ").strip()
                inside_list = False
                buffer = []
                continue

            # Detect any opening of a list
            if "[" in line and not inside_list and current_key:
                inside_list = True
                buffer = [line[line.find("["):]]  # keep only from the first `[`
                continue

            # Collect lines until the list is closed
            if inside_list:
                buffer.append(line)
                if "]" in line:
                    inside_list = False
                    try:
                        full_text = "\n".join(buffer)
                        tickers = ast.literal_eval(full_text)
                        request_blocks[current_key] = tickers
                    except Exception as e:
                        print(f"⚠️ Failed to parse tickers for {current_key}: {e}")
                        print("Block was:\n", full_text)
                    buffer = []

    return request_blocks

data = extract_all_ticker_lists(os.path.join("..", "data/chatgpt_tickers.txt"))

In [ ]:
df = pd.DataFrame([
    {
        "request": k,
        "n": int(k.split("|")[0].split("=")[1].strip()),
        "tickers": v,
        "count": len(v)
    }
    for k, v in data.items()
])

df_exploded = df.explode("tickers").rename(columns={"tickers": "ticker"})

# Preview
print(df.head())
print(df_exploded["ticker"].value_counts().head(10))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

for i in choices:
    df = df_exploded[df_exploded["n"] == i]
    ticker_counts = df['ticker'].value_counts()

    top_n = i
    top_tickers = ticker_counts.head(top_n)
    other_tickers = ticker_counts.iloc[top_n:]

    # Plot
    plt.figure(figsize=(14, 6))

    plt.bar(top_tickers.index, top_tickers.values, color='green', label=f"Top {i} stocks")
    plt.bar(other_tickers.index, other_tickers.values, color='red', hatch='//', label='Other stocks')

    plt.xticks(rotation=90, ha="right", fontsize=5)
    plt.xlabel("Stock Tickers")
    plt.ylabel("Frequency")
    plt.legend()
    plt.title(f"Universe Selected by GPT-4 for the Top {i} Stocks")

    plt.savefig(os.path.join('..',f"figs/top_{i}.png"))

In [ ]:
def generate_chatgpt_insight(ticker):
    """Generate a prompt and request from OpenAI about the sentiment of a given company."""

    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    prompt = (
        f"Given the following stock {ticker}, "
        "please write a report on the current state of the company. "
        "Please be clear on when you collected the most recent data. "
        "What are the key insights of each company? "
        "Please highlight the positives and negatives in an objective way for fair sentiment analysis."
        "Include any information from their financial reports or news."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def process_top_stocks(df, top_values=[15, 30, 45, 60], output_dir="sentiment"):
    """Process top X stocks for each choice and save insights to text files."""
    # Ensure output directory exists
    os.makedirs(f"../data/{output_dir}", exist_ok=True)

    # Collect all top stocks across the specified top values
    unique_stocks = set()
    for top_x in top_values:
        # Get top X stocks for each choice and add to the set
        top_stocks = df.groupby("request")["ticker"].value_counts().groupby(level=0).nlargest(top_x).reset_index(level=0, drop=True)
        unique_stocks.update(top_stocks.index.get_level_values(1))

    for ticker in tqdm.tqdm(unique_stocks, desc="Generating Insights"):
        file_path = os.path.join(f"../data/{output_dir}", f"{ticker}.txt")
        
        if os.path.exists(file_path):
            continue
        
        try:
            insight = generate_chatgpt_insight(ticker)
        except Exception as e:
            insight = f"Error generating insight: {str(e)}"
        
        # Save the insight to the file
        with open(file_path, "w") as file:
            file.write(f"Ticker: {ticker}\n")
            file.write(f"Insight:\n{insight}\n")
    
    print(f"Insights saved in the '{output_dir}' directory.")

process_top_stocks(df_exploded, top_values=[15, 30, 45, 60])

In [ ]:
def download_stock_data(ticker, output_dir="market", retries=5):
    file_path = os.path.join(f"../data/{output_dir}", f"{ticker}.csv")
    if os.path.exists(file_path):
        return f"{ticker} - Already downloaded."

    attempt = 0
    while attempt < retries:
        try:
            data = yf.download(ticker, start="2010-01-01", end="2025-01-01")
            if data.empty:
                return f"{ticker} - No data available."
            
            # Save the data to file
            data.to_csv(file_path)
            return f"{ticker} - Downloaded successfully."
        
        except yf.YFRateLimitError:
            attempt += 1
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"{ticker} - Rate limited. Retrying in {wait_time}s...")
            time.sleep(wait_time)
        except Exception as e:
            return f"{ticker} - Error: {str(e)}"
    
    return f"{ticker} - Failed after {retries} retries."

In [ ]:
def process_top_stocks_data(df, top_values=[15, 30, 45, 60], output_dir="market"):
    """Process top X stocks for each choice and save insights to text files."""
    # Ensure output directory exists
    os.makedirs(f"../data/{output_dir}", exist_ok=True)

    unique_stocks = set()
    for top_x in top_values:
        top_stocks = df.groupby("request")["ticker"].value_counts().groupby(level=0).nlargest(top_x).reset_index(level=0, drop=True)
        unique_stocks.update(top_stocks.index.get_level_values(1))
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(download_stock_data, ticker, output_dir): ticker for ticker in unique_stocks}
        
        for future in tqdm.tqdm(as_completed(futures), total=len(futures), desc="Downloading data"):
            ticker = futures[future]
            try:
                result = future.result()
                print(result)
            except Exception as e:
                print(f"{ticker} - Unexpected Error: {str(e)}")

process_top_stocks_data(df_exploded, top_values=[15, 30, 45, 60])

In [ ]:
def generate_chatgpt_allocation(stocks):
    """Generates a theoretical model portfolio insight for the given stocks using ChatGPT."""

    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    prompt = (
        f"Assume you're designing a theoretical model portfolio from these S&P500 stocks: {', '.join(stocks)}. "
        "Provide an example of how you might distribute the weightage of these stocks (normalized, i.e., weights should add up to 1.00) in the portfolio "
        "to potentially outperform the S&P500 index. Also mention the underlying strategy or logic which you used to assign these weights. "
        "Please provide any additional rationale when deciding the allocation."
    )
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def save_chatgpt_responses(df, top_values=[15, 30, 45, 60], output_dir="chatgpt_responses"):
    os.makedirs(f"../data/{output_dir}", exist_ok=True)
    
    # Collect top X stocks for each specified value
    for n in top_values:
        df_filtered = df[df['request'].str.contains(f"N={n}")]
        top_n_stocks = df_filtered['ticker'].value_counts().head(n).index.tolist()

        response = generate_chatgpt_allocation(list(top_n_stocks))

        # Save the response to a text file
        file_path = os.path.join(f"../data/{output_dir}", f"top_{n}_stocks_portfolio.txt")
        with open(file_path, "w") as file:
            file.write(f"Top {n} Stocks: {', '.join(top_n_stocks)}\n\n")
            file.write(response)
        
    print(f"ChatGPT responses saved in the '{output_dir}' directory.")
save_chatgpt_responses(df_exploded)

In [ ]:
df_exploded

In [ ]:
csv_directory = "../data/market" 

# Define the n values for the four groups
n_values = [15, 30, 45, 60]

def plot_top_n_stocks(df, csv_directory, n_values):
    for n in n_values:
        os.makedirs(f"../figs/stock_price", exist_ok=True)
        df_filtered = df[df['request'].str.contains(f"N={n}")]
        
        # Calculate the top N stocks for this group
        top_n_stocks = df_filtered['ticker'].value_counts().head(n).index.tolist()

        plt.figure(figsize=(14, 8))
        plotted_stocks = 0  

        for ticker in top_n_stocks:
            csv_path = os.path.join(csv_directory, f"{ticker}.csv")
            if os.path.exists(csv_path):
                stock_data = pd.read_csv(
                    csv_path, 
                    skiprows=2, 
                    names=["Date", "Close", "High", "Low", "Open", "Volume"],
                    dtype={
                        "Date": str, 
                        "Close": float, 
                        "High": float, 
                        "Low": float, 
                        "Open": float, 
                        "Volume": float
                    },
                    na_values=["", "NaN", "N/A", "null"]
                )

                # Dropping rows with invalid dates or NaN values in essential columns
                stock_data = stock_data.dropna(subset=["Date", "Close"])

                # Converting Date column to datetime 
                stock_data["Date"] = pd.to_datetime(stock_data["Date"], errors="coerce")
                stock_data = stock_data.dropna(subset=["Date"])

                # Setting Date as index
                stock_data.set_index("Date", inplace=True)
                if 'Close' in stock_data.columns:
                    plt.plot(stock_data['Close'], label=ticker)
                    plotted_stocks += 1
     
        if plotted_stocks > 0:
            plt.title(f"Top {n} Stocks - Price vs Time")
            plt.xlabel("Date")
            plt.ylabel("Stock Price (USD)")
            if n==60 or n==45:
                plt.legend(loc="upper left", fontsize=8, ncol=2)
            else:
                plt.legend(loc="upper left", fontsize=8)
            plt.tight_layout()
            plt.savefig(f"../figs/stock_price/top_{n}.png")
        else:
            print(f"No valid data for Top {n} Stocks.\n")

# Call the function
plot_top_n_stocks(df_exploded, csv_directory, n_values)

### Pure MVO

In [ ]:
def calculate_returns_df(n):
    df_filtered = df_exploded[df_exploded['request'].str.contains(f"N={n}")]
    
    top_n_stocks = df_filtered['ticker'].value_counts().head(n).index.tolist()

    stock_data = {}

    for ticker in top_n_stocks:
        file_name =  f"../data/market/{ticker}.csv"
        try:
            df_raw = pd.read_csv(file_name, skiprows=[1, 2])
            
            if ticker == "UNH":
                df_raw = df_raw.iloc[:, [0, 2, 5, 8, 11, 14]]
            else:
                df_raw = df_raw.iloc[:, :6]
        
            df_raw.columns = ["Date", "Close", "High", "Low", "Open", "Volume"]
            df_raw.dropna(how="all", inplace=True) 

            # Converting the Date column to datetime format
            df_raw["Date"] = pd.to_datetime(df_raw["Date"], errors="coerce")

            # Converting the Close column to float 
            df_raw["Close"] = pd.to_numeric(df_raw["Close"], errors="coerce")

            df_clean = df_raw.set_index("Date")
            df_clean["Returns"] = df_clean["Close"].pct_change(fill_method=None) * 100
            stock_data[ticker] = df_clean["Returns"]
        except Exception as e:
            print(ticker, e)

    returns_df = pd.DataFrame()

    for stock, df in stock_data.items():
        returns_df[stock] = df

    return returns_df

In [ ]:
top_15_returns = calculate_returns_df(15)
top_30_returns = calculate_returns_df(30)
top_45_returns = calculate_returns_df(45)
top_60_returns = calculate_returns_df(60)

In [ ]:
top_60_returns.head()

### 1. Equally Weighted Portfolio

In [ ]:
os.makedirs('../figs/backtests/equal', exist_ok=True)

def compute_metrics(returns, n):
	# Ensure returns are in decimal form and handle NaN values
    returns = returns / 100  
    returns.fillna(0, inplace=True)  

    # Calculate the equally weighted portfolio returns
    equal_weights = 1 / returns.shape[1]
    portfolio_returns = (returns * equal_weights).mean(axis=1)

    # Calculate cumulative returns
    cumulative_returns = (1 + portfolio_returns).cumprod()

    # Plot the cumulative returns
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_returns, label='Equally Weighted Portfolio')
    plt.title(f"Equally Weighted Portfolio Backtest n={n}")
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.savefig(f'../figs/backtests/equal/top_{n}.png')

    # Performance Metrics
    total_return = cumulative_returns.iloc[-1] - 1
    annualized_return = (cumulative_returns.iloc[-1]) ** (252 / len(cumulative_returns)) - 1
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility

    # Display metrics
    print(f'Total Return: {total_return:.2%}')
    print(f'Annualized Return: {annualized_return:.2%}')
    print(f'Annualized Volatility: {annualized_volatility:.2%}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

In [ ]:
compute_metrics(top_15_returns, 15)

In [ ]:
compute_metrics(top_30_returns, 30)

In [ ]:
compute_metrics(top_45_returns, 45)

In [ ]:
compute_metrics(top_60_returns, 60)

# GPT Weighted

In [ ]:
os.makedirs('../figs/backtests/gpt', exist_ok=True)

def compute_gpt_returns(returns, n, custom_weights):
    # Ensure returns are in decimal form and handle NaN values
    returns = returns / 100  
    returns.fillna(0, inplace=True)  

    dropped_columns = set(custom_weights.keys()) - set(returns.columns)
    if dropped_columns:
        print(f"Missing columns in returns data: {dropped_columns}")

    aligned_weights = {ticker: weight for ticker, weight in custom_weights.items() if ticker in returns.columns}

    portfolio_returns = (returns[list(aligned_weights.keys())] * list(aligned_weights.values())).sum(axis=1)

    cumulative_returns = (1 + portfolio_returns).cumprod()

    # Plot the cumulative returns
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_returns, label='Custom Weighted Portfolio')
    plt.title(f'GPT Weighted Portfolio Backtest n={n}')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.savefig(f"../figs/backtests/gpt/top_{n}.png")

    # Performance Metrics
    total_return = cumulative_returns.iloc[-1] - 1
    annualized_return = (cumulative_returns.iloc[-1]) ** (252 / len(cumulative_returns)) - 1
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility

    # Display metrics
    print(f'Total Return: {total_return:.2%}')
    print(f'Annualized Return: {annualized_return:.2%}')
    print(f'Annualized Volatility: {annualized_volatility:.2%}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

In [ ]:
custom_15_weights = {
        'MSFT': 0.130,
        'AAPL': 0.120,
        'NVDA': 0.105,
        'AMZN': 0.085,
        'GOOGL': 0.080,
        'META': 0.060,
        'AVGO': 0.055,
        'LLY': 0.075,
        'UNH': 0.050,
        'JPM': 0.040,
        'MA': 0.040,
        'V': 0.040,
        'COST': 0.045,
        'LIN': 0.030,
        'HD': 0.030
    }

compute_gpt_returns(top_15_returns, 15, custom_15_weights)

In [ ]:
custom_30_weights = {
    'AAPL': 0.06,
    'MSFT': 0.06,
    'GOOGL': 0.06,
    'NVDA': 0.07,
    'AMZN': 0.06,
    'META': 0.06,
    'AVGO': 0.05,
    'UNH': 0.03,
    'LLY': 0.05,
    'COST': 0.04,
    'HD': 0.03,
    'NKE': 0.02,
    'LIN': 0.02,
    'ADBE': 0.04,
    'V': 0.03,
    'MA': 0.03,
    'MRK': 0.02,
    'TMO': 0.03,
    'JNJ': 0.03,
    'WMT': 0.04,
    'PGR': 0.02,
    'CRM': 0.03,
    'TSLA': 0.04,
    'SCHW': 0.02,
    'PEP': 0.03,
    'ORCL': 0.03,
    'PG': 0.03,
    'JPM': 0.03,
    'SPGI': 0.02
}

compute_gpt_returns(top_30_returns, 30, custom_30_weights)

In [ ]:
custom_45_weights = {
	'AAPL': 0.07,
    'MSFT': 0.08,
    'GOOGL': 0.065,
    'AMZN': 0.06,
    'NVDA': 0.07,
    'META': 0.045,
    'AVGO': 0.035,
    'AMD': 0.02,
    'ADBE': 0.015,
    'ORCL': 0.01,
    'CRM': 0.01,
    'NFLX': 0.015,
    'TSLA': 0.02,
    'UNH': 0.025,
    'LLY': 0.03,
    'JNJ': 0.02,
    'MRK': 0.015,
    'REGN': 0.01,
    'VRTX': 0.01,
    'TMO': 0.01,
    'ABBV': 0.01,
    'CAT': 0.015,
    'DE': 0.01,
    'HON': 0.01,
    'LMT': 0.01,
    'LIN': 0.01,
    'MS': 0.01,
    'JPM': 0.015,
    'BAC': 0.005,
    'GS': 0.005,
    'SCHW': 0.005,
    'MA': 0.025,
    'V': 0.02,
    'PEP': 0.015,
    'KO': 0.01,
    'PG': 0.015,
    'WMT': 0.015,
    'COST': 0.02,
    'HD': 0.01,
    'TJX': 0.01,
    'CMG': 0.01,
    'SBUX': 0.01,
    'NKE': 0.01,
    'NEE': 0.01
}

compute_gpt_returns(top_45_returns, 45, custom_45_weights)

In [ ]:
custom_60_weights = {
	'AAPL': 0.05, 'MCD': 0.02, 'UNP': 0.015, 'PLD': 0.01,
    'MSFT': 0.05, 'TMO': 0.015, 'LMT': 0.01, 'SLB': 0.01,
    'CAT': 0.015, 'ADBE': 0.02, 'HON': 0.015, 'EOG': 0.01,
    'MS': 0.015, 'COST': 0.01, 'PG': 0.02, 'ABBV': 0.02,
    'MCD': 0.02, 'UNH': 0.02, 'BLK': 0.015, 'BAC': 0.01,
    'TMO': 0.015, 'AMD': 0.015, 'PEP': 0.015, 'ACN': 0.02,
    'ADBE': 0.02, 'JPM': 0.02, 'CRM': 0.015, 'SBUX': 0.01,
    'COST': 0.01, 'AVGO': 0.02, 'NFLX': 0.015, 'CVX': 0.01,
    'UNH': 0.02, 'NVDA': 0.03, 'INTU': 0.015, 'ABT': 0.01,
    'AMD': 0.015, 'LLY': 0.03, 'HD': 0.015, 'CL': 0.005,
    'JPM': 0.02, 'GOOGL': 0.03, 'DE': 0.01, 'RTX': 0.01,
    'AVGO': 0.02, 'AMZN': 0.02, 'LOW': 0.01, 'CSCO': 0.01,
    'NVDA': 0.03, 'META': 0.02, 'VRTX': 0.01, 'TSLA': 0.02,
    'LLY': 0.03, 'UNP': 0.015, 'GS': 0.01, 'UPS': 0.01,
    'GOOGL': 0.03, 'LMT': 0.01, 'SCHW': 0.01, 'MA': 0.015,
    'AMZN': 0.02, 'HON': 0.015, 'REGN': 0.01, 'NEE': 0.015,
    'META': 0.02, 'PG': 0.02, 'JNJ': 0.02, 'LIN': 0.01,
    'ORCL': 0.015, 'NKE': 0.01, 'XOM': 0.01, 'WMT': 0.02,
    'REGN': 0.01, 'V': 0.015, 'MRK': 0.01, 'KO': 0.015
}

compute_gpt_returns(top_60_returns, 60, custom_60_weights)

In [ ]:
r = 0.08

w = cp.Variable(15)
objective = cp.Minimize(cp.quad_form(w, sigma))
constraints = [
    cp.sum(w) == 1,           
    w @ mu >= r,
	w >= 0,      
]

problem = cp.Problem(objective, constraints)

try:
    problem.solve(solver = cp.CLARABEL)
    print ("Optimal portfolio")
    print ("----------------------")
    for s in range(len(mu)):
        print (" Investment in {} : {}% of the portfolio".format(s + 1,round(100*w.value[s],2)))
    print ("----------------------")
    print (f"Expected return = {(w.value @ mu)}%")
    print (f"Expected risk = {np.sqrt(w.value.T @ sigma @ w.value)}%")
except Exception as e:
    print(e)